In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from my_notion_companion.retriever import SelfQueryAgent, BasicRetriever

In [4]:
from langchain_community.llms import LlamaCpp

import tomllib

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+'Qwen-7B-Chat.Q4_K_M.gguf',
    name='Qwen/Qwen-7B-Chat', 
    **_CONFIGS['llm']
)

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 259 tensors from /Users/fred/Documents/models/Qwen-7B-Chat.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen
llama_model_loader: - kv   1:                               general.name str              = Qwen
llama_model_loader: - kv   2:                        qwen.context_length u32              = 32768
llama_model_loader: - kv   3:                           qwen.block_count u32              = 32
llama_model_loader: - kv   4:

In [5]:
import re

with open('../data/test_cases.txt') as f:
    test_cases_raw = f.readlines()

test_cases_raw = "".join(test_cases_raw[0:-1:2]).split('问：')[1:]
test_cases_raw = [re.split(r'\n答：|\n资料：', x) for x in test_cases_raw]
test_cases = [{'q': x[0], 'a': x[1], 'docs': x[2:]} for x in test_cases_raw]

test_cases[:3]

[{'q': '什么是我国第一部编年史著作？', 'a': '《左传》。', 'docs': ['附：《左传》是我国第一部编年史著作。\n']},
 {'q': '什么是我国第一部编年国别史？', 'a': '《国语》。', 'docs': ['附：《国语》是我国第一部编年国别史。\n']},
 {'q': '“寡人之于国也”下一句是什么？来自哪里？',
  'a': '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  'docs': ['梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
   '《寡人之于国也》（孟子）\n']}]

In [6]:
len(test_cases)

37

In [7]:
from tqdm import tqdm

def evaluate_retriever(test_cases, retriever):
    score_list = list()

    for case in tqdm(test_cases):
        docs_retrieved = retriever.invoke(case['q'])
        contains_all_ref = list()
        for ref in case['docs']:
            contains_all_ref.append(any([ref in x.page_content for x in docs_retrieved]))
        
        score_list.extend(contains_all_ref)

    return sum(score_list)/len(score_list), score_list

In [10]:
%%time

stats, score_list = evaluate_retriever(test_cases,  BasicRetriever(llm, _CONFIGS, _TOKENS))

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in simil

CPU times: user 1.13 s, sys: 118 ms, total: 1.25 s
Wall time: 4.07 s


In [11]:
stats

0.0625

In [12]:
%%time

stats, score_list = evaluate_retriever(test_cases,  SelfQueryAgent(llm, _CONFIGS, _TOKENS))


llama_print_timings:        load time =    8357.14 ms
llama_print_timings:      sample time =      91.75 ms /   256 runs   (    0.36 ms per token,  2790.25 tokens per second)
llama_print_timings: prompt eval time =    8354.64 ms /  1134 tokens (    7.37 ms per token,   135.73 tokens per second)
llama_print_timings:        eval time =   12966.16 ms /   255 runs   (   50.85 ms per token,    19.67 tokens per second)
llama_print_timings:       total time =   23012.52 ms /  1389 tokens
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 

CPU times: user 26min 16s, sys: 3min 26s, total: 29min 43s
Wall time: 39min 10s


In [13]:
stats

0.0625

In [8]:
import pickle
with open('../data/notion_offline.pkl', 'rb') as f:
    docs_from_notion = pickle.load(f)

In [12]:
docs_from_notion['写作'][0]

Document(page_content='师说\n\n序\n我怀疑……我得了甲流。\n\n\n（一）\n此地有崇山峻岭茂林修竹\n是能读三坟五典八索九丘\n\n我有三本教辅。\n《高效阅读与写作》、《高中古典诗歌教程》、《高考作文经典材料百变通》。\n我有三札摘抄。\n“写得好”“阅”“这是我近来看到的最好的诗。继续写下去，加油！”\n我记得三句话。\n“人无权反对他不了解的事物。”\n“被选择的真实等于谎言。”\n“人生就是奋斗。”\n\n\n（二）\n闻姊家有阁子，且何谓阁子也？\n\n0.0\n夏令营时每天都去学术报告厅听课。\n脑子里一直是奥林匹克、竞赛金牌、优秀的学长和高中的目标。同学们都好强啊…还要洗衣服…今天晚上又没有水果吃……\n好满、好压抑，人生好失败。\n然后突然出现这么一个人。他没有谈在七中成长的十点建议。没有讲成为优秀的人的十种习惯。没有说自主学习的重要性、也不教导你做最好的自己。\n他跟你讲论语。\n八佾第三。孔子对曰：\n“君使臣以礼，臣事君以忠。”\n我知道啊，我知道啊。所以您永远是我们的罗皇。\n\n1.0\n第一次站起来回答问题的时候我坐在8-1。那时我们在讲卞之琳。\n“你站在桥上看风景，看风景人在楼上看你。”\n——杨同学，请你来分析一下这首这句话的哲理。\n血压一下子生得好高。\n语文老师请我回答问题！他知道我名字！\n……\n接下来就是一片空白。因为我压根儿就在开小差。\n——呃，一个事物在某种条件下可以转化为另一种事物……大概是，这样吧。\n皇上笑笑让我坐下，所以你这是指A=B于是B=A么？\n侨妹站起来了。\n——这是主客体之间的相互转化。\n感觉自己瞬间就低端了。\n拜托啊せんせい，您可千万别放弃我。\n我真的是很崇拜您的呢。\n\n2.0\n新妇初来时，小姑始扶床；今日被驱遣，小姑如我长。\n原来兰芝只有这么高啊。\n惟草木之零落兮，恐美人之迟暮。\n原来有人说屈原是XXX啊。\n刘少奇被钉在历史的耻辱柱上；\n没钉稳，结果掉了下来。\n原来这就是近代史啊。\n科学家张衡，在史书上是个官员。\n原来这就是中国啊。\n“奇穿偶不变，符号看象限。”\n——原来这就是我们的皇后啊！\n\n3.0\n那一堂课的内容是《杜十娘怒沉百宝箱》。\n因为无聊和不想睡觉，我们班那个没有脖颈的男生写了一首七绝。\n因为无聊和挑战自我

In [18]:
docs = list()
for k, v in docs_from_notion.items():
    docs.extend(v)

In [19]:
len(docs)

225

In [22]:
from langchain.retrievers import BM25Retriever

retriever = BM25Retriever.from_documents(docs)

In [23]:
%%time

stats, score_list = evaluate_retriever(test_cases,  retriever)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 7400.18it/s]

CPU times: user 7.71 ms, sys: 14.7 ms, total: 22.4 ms
Wall time: 59 ms


In [24]:
stats # doesn't work well with Chinese

0.0